In [1]:
import pandas as pd
pd.set_option('display.max_columns', 50)
import MetaTrader5 as mt5
from datetime import datetime, timezone, timedelta
import pytz
import os
import json
import traceback
import time

import sys
path_root = "D:/Documentos/Erik/TDR/TDR-Forex/"
sys.path.append(path_root)
from utils.data import get_symbol_details
from utils.indicators import get_EMAs
from utils.strategies import load_strategy_parameters, get_strategy, CalcLotSize
from utils.mt5 import *

In [2]:
# Funció per netejar la línia actual
def clear_line():
    print('\r' + ' ' * 100, end='', flush=True)

# Funció per imprimir amb actualització
def print_update(message, end_line=False):
    if end_line:
        print(message)
    else:
        print('\r' + message, end='', flush=True)

In [4]:
# Configuració inicial
symbol = "EURUSD"; MagicNumber = 1006;
strategy_params = load_strategy_parameters(MagicNumber, path_root)
print("PARÀMETRES DEL SISTEMA:")
print(json.dumps(strategy_params, indent=2, ensure_ascii=False))
# Definir paràmetres constants de l'estratègia
StrategyName = strategy_params["Name"]
DynamicLotSize = strategy_params["DynamicLotSize"]
EquityPercent = strategy_params["EquityPercent"]
FixedLotSize = strategy_params["FixedLotSize"]
MaxOpenTrades = strategy_params["MaxOpenTrades"]
MaxMinutesOpenTrades = 1
MinBetweenTrades = 1
TPPips_short = strategy_params["TP_short"]
TPPips_long = strategy_params["TP_long"]
SLPips_long = strategy_params["SL_long"]
SLPips_short = strategy_params["SL_short"]
server_timezone = get_server_timezone()
last_status = ""

PARÀMETRES DEL SISTEMA:
{
  "Name": "Mitjanes",
  "TimeFrame": "M1",
  "NBars": 200,
  "DynamicLotSize": false,
  "EquityPercent": 2.5,
  "FixedLotSize": 0.01,
  "MaxOpenTrades": 5,
  "MaxMinutesOpenTrades": 720,
  "MinBetweenTrades": 5,
  "TP_short": 100,
  "TP_long": 100,
  "SL_long": 250,
  "SL_short": 250
}


In [5]:
def get_parameters(server_timezone, MagicNumber, symbol):
    now_server_time = datetime.now(server_timezone)
        
    # Obtenir posicions i deals
    positions = get_positions_df(MagicNumber=MagicNumber, symbol=symbol)
    n_positions = len(positions)
        
    deals = get_deals_df(MagicNumber=MagicNumber, symbol=symbol)
        
    # Calcular temps des de última entrada/sortida
    if not positions.empty:
        last_entry_time = positions['time'].max()
        if last_entry_time.tzinfo is None:
            last_entry_time = server_timezone.localize(last_entry_time)
    else:
        last_entry_time = pd.to_datetime("2000-01-01").tz_localize(server_timezone)
        
    if not deals.empty and not deals[deals['entry'] == 1].empty:
        exit_time_prev = deals[deals['entry'] == 1]['time'].max()
        if exit_time_prev.tzinfo is None:
            exit_time_prev = server_timezone.localize(exit_time_prev)
    else:
        exit_time_prev = pd.to_datetime("2000-01-01").tz_localize(server_timezone)
        
    minutes_elapsed_since_last_entry = (now_server_time - last_entry_time).total_seconds() / 60
    minutes_elapsed_since_last_exit = (now_server_time - exit_time_prev).total_seconds() / 60
    minutes_elapsed = min(minutes_elapsed_since_last_entry, minutes_elapsed_since_last_exit)

    return now_server_time, positions, n_positions, minutes_elapsed

def get_account_equity():
    """
    Obté l'equity actual del compte de MT5
    """
    if not mt5.initialize():
        print(f"Error inicialitzant MT5: {mt5.last_error()}")
        return None
    
    try:
        # Obtenir informació del compte
        account_info = mt5.account_info()
        
        if account_info is None:
            print(f"Error obtenint info del compte: {mt5.last_error()}")
            return None
        
        # Retornar l'equity
        return account_info.equity
        
    except Exception as e:
        print(f"Error inesperat: {e}")
        return None
    finally:
        mt5.shutdown()

In [ ]:
while True:
    Slippage = 10
    try:
        AccountEquity = get_account_equity()
        now_server_time, positions, n_positions, minutes_elapsed = get_parameters(server_timezone, MagicNumber, symbol)
        data = get_strategy(symbol, MagicNumber, path_root, True)
        last_data = data.iloc[-1]
        signal = last_data["signal"]
        cond_close_long = last_data["cond_close_long"]
        cond_close_short = last_data["cond_close_short"]
        
        # Actualitzar informació en pantalla
        current_status = f"{now_server_time.strftime('%H:%M:%S')} | Pos: {n_positions}/{MaxOpenTrades} | Min: {minutes_elapsed:.0f} | Senyal: {signal} | Tancar pos. llargues: {cond_close_long} | Tancar pos. curtes: {cond_close_short} | Equity: {AccountEquity:.2f}"
        if current_status != last_status:
            print_update(current_status)
            last_status = current_status

        # Executar operacions - COMPRES
        if (signal == 1) and (n_positions < MaxOpenTrades) and (minutes_elapsed > MinBetweenTrades):
            LotSize = CalcLotSize(DynamicLotSize, AccountEquity, EquityPercent, SLPips_long, FixedLotSize)
            print_update(f"Obrint COMPRA - Lots: {LotSize:.3f}", end_line=True)
            ticket = OpenBuyOrder(symbol, LotSize, Slippage, MagicNumber, StrategyName, argSLPips=SLPips_long/10, argTPPips=TPPips_long/10)
            if ticket:
                print_update(f"COMPRA oberta - Ticket: {ticket}", end_line=True)

        # Executar operacions - VENDES
        elif (signal == -1) and (n_positions < MaxOpenTrades) and (minutes_elapsed > MinBetweenTrades):
            LotSize = CalcLotSize(DynamicLotSize, AccountEquity, EquityPercent, SLPips_short, FixedLotSize)
            print_update(f"Obrint VENDA - Lots: {LotSize:.3f}", end_line=True)
            ticket = OpenSellOrder(symbol, LotSize, Slippage, MagicNumber, StrategyName, argSLPips=SLPips_short/10, argTPPips=TPPips_short/10)
            if ticket:
                print_update(f"VENDA oberta - Ticket: {ticket}", end_line=True)

        # Tancar posicions llargues
        if cond_close_long == 1 and not positions.empty:
            tickets_BUY_positions = positions[positions['type'] == 0]['ticket'].tolist()
            if tickets_BUY_positions:
                print_update(f"Tancant {len(tickets_BUY_positions)} posicions llargues", end_line=True)
                for ticket in tickets_BUY_positions:
                    result = CloseOrderByTicket(ticket, Slippage)
                    if result:
                        print_update(f"Posició llarga tancada - Ticket: {ticket}", end_line=True)

        # Tancar posicions curtes
        if cond_close_short == 1 and not positions.empty:
            tickets_SELL_positions = positions[positions['type'] == 1]['ticket'].tolist()
            if tickets_SELL_positions:
                print_update(f"Tancant {len(tickets_SELL_positions)} posicions curtes", end_line=True)
                for ticket in tickets_SELL_positions:
                    result = CloseOrderByTicket(ticket, Slippage)
                    if result:
                        print_update(f"Posició curta tancada - Ticket: {ticket}", end_line=True)

        # Esperar abans de la propera iteració
        time.sleep(5)  # Reduït a 5 segons per actualitzacions més freqüents
        
    except KeyboardInterrupt:
        print_update("\n Aturat per l'usuari", end_line=True)
        break
        
    except Exception as e:
        error_msg = f"Error: {e}"
        print_update(error_msg, end_line=True)
        time.sleep(60)

Obrint COMPRA - Lots: 0.01077 | Senyal: 1.0 | Tancar pos. llargues: 0.0 | Tancar pos. curtes: 1.0 | Equity: 1000.40
Obrint ordre BUY: EURUSD 0.01 lots
Preu: 1.15962
SL: 1.15712 (25.0 pips)
TP: 1.16062 (10.0 pips)
Ordre executada correctament!
   Order: 4436590982, Deal: 4382204152
   SL: 1.15712
   TP: 1.16062
COMPRA oberta - Ticket: 4436590982
Obrint COMPRA - Lots: 0.01045 | Senyal: 1.0 | Tancar pos. llargues: 0.0 | Tancar pos. curtes: 1.0 | Equity: 1000.83
Obrint ordre BUY: EURUSD 0.01 lots
Preu: 1.16018
SL: 1.15768 (25.0 pips)
TP: 1.16118 (10.0 pips)
Ordre executada correctament!
   Order: 4437234599, Deal: 4382847859
   SL: 1.15768
   TP: 1.16118
COMPRA oberta - Ticket: 4437234599
Tancant 2 posicions llargues9 | Senyal: 0.0 | Tancar pos. llargues: 1.0 | Tancar pos. curtes: 0.0 | Equity: 998.340
Ticket 4436590982 closed successfully
Ticket 4437234599 closed successfully
Error: [Errno 2] No such file or directory: 'D:/Documentos/Erik/TDR/TDR-Forex/output/Forex/strategies\\dict_strate